In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
lv = LV.from_pyvista_read("../sample_files/lvtetmesh.vtk")
# lv.smooth_surface(n_iter=500)
lv.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)
lv.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value,
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(position=(49999.999999…

In [3]:
# transform point region ids into cell ids at surface level
cellregionIdsSurf = lv.transform_point_data_to_cell_data(LV_MESH_DATA.SURFS_DETAILED.value, surface=True)
# combine volumetric mesh with surface mesh
mesh = lv.mesh.copy()
mesh = mesh.merge(lv.get_surface_mesh())
# adjust regions to include both surface and volume (with zeros)
cellregionIds = np.hstack((cellregionIdsSurf, np.zeros(mesh.n_cells- len(cellregionIdsSurf))))
# add gmsh data
mesh.clear_data() # for some reason, no other info is accepted when loading in ldrb
mesh.cell_data["gmsh:physical"] = cellregionIds
mesh.cell_data["gmsh:geometrical"] = cellregionIds
# save using meshio (I did not test other gmsh formats and binary files.)
pv.save_meshio("../sample_files/lvtetmesh.msh", mesh, file_format="gmsh22", binary=False)

In [4]:
import ldrb

In [5]:
# Last argument here is the markers, but these are not used
mesh, ffun, _ = ldrb.gmsh2dolfin(
    "../sample_files/lvtetmesh.msh",
    unlink=False,
)
# Run this first in serial and exit here
# exit()

In [25]:
lv.mesh.points

pyvista_ndarray([[ -3.87907733,  12.52942467, -19.55683257],
                 [ -3.01636916,  13.5763117 , -19.82178605],
                 [ -2.7924061 ,  12.583106  , -19.41994991],
                 ...,
                 [ 29.51456854, -38.22710626, -23.7777292 ],
                 [ 23.35306941, -25.0733027 ,  15.30362608],
                 [ 21.7924402 , -15.76549157, -45.14888886]])

In [16]:
mesh.coordinates()

array([[ 38.11470565, -12.63239147, -38.03530351],
       [ 38.1706562 , -13.23636135, -38.21822321],
       [ 38.65349265, -12.76732667, -37.86798067],
       ...,
       [ 37.36369972,  31.72167606,   5.04356019],
       [ 23.99444147, -29.0357094 ,   0.56204273],
       [  0.82010905,  11.72971847, -24.40812559]])

In [44]:
from project_heart.utils.cloud_ops import relate_closest

In [55]:
map_from_mesh_to_pts = relate_closest(lv.mesh.points, mesh.coordinates())[0]
map_from_mesh_to_pts

array([[     0,  52193],
       [     1,  52194],
       [     2,  52195],
       ...,
       [149633,  30266],
       [149634,  51746],
       [149635,  14957]])

In [17]:
for i in np.unique(lv.mesh.get_array(LV_MESH_DATA.SURFS_DETAILED.value)):
  print(LV_SURFS(i))

LV_SURFS.OTHER
LV_SURFS.ENDO
LV_SURFS.EPI
LV_SURFS.ENDO_AORTIC
LV_SURFS.EPI_AORTIC
LV_SURFS.BORDER_AORTIC
LV_SURFS.ENDO_MITRAL
LV_SURFS.EPI_MITRAL
LV_SURFS.BORDER_MITRAL
LV_SURFS.ENDO_AM_INTERCECTION
LV_SURFS.EPI_AM_INTERCECTION


In [18]:
markers = {
  "epi": LV_SURFS.EPI.value, 
  "lv": LV_SURFS.ENDO.value, 
  "base": LV_SURFS.MITRAL.value
  }


ffun.array()[ffun.array() == LV_SURFS.EPI_AM_INTERCECTION] = LV_SURFS.EPI.value
ffun.array()[ffun.array() == LV_SURFS.EPI_AORTIC] = LV_SURFS.EPI.value
ffun.array()[ffun.array() == LV_SURFS.EPI_MITRAL] = LV_SURFS.EPI.value

ffun.array()[ffun.array() == LV_SURFS.ENDO_AM_INTERCECTION] = LV_SURFS.ENDO.value
ffun.array()[ffun.array() == LV_SURFS.ENDO_AORTIC] = LV_SURFS.ENDO.value
ffun.array()[ffun.array() == LV_SURFS.ENDO_MITRAL] = LV_SURFS.ENDO.value

ffun.array()[ffun.array() == LV_SURFS.BORDER_AORTIC] = LV_SURFS.MITRAL.value
ffun.array()[ffun.array() == LV_SURFS.BORDER_MITRAL] = LV_SURFS.MITRAL.value

In [19]:
fiber_space = "P_1"

fiber, sheet, sheet_normal = ldrb.dolfin_ldrb(
    mesh=mesh,
    fiber_space=fiber_space,
    ffun=ffun,
    markers=markers,
    alpha_endo_lv=60,  # Fiber angle on the endocardium
    alpha_epi_lv=-60,  # Fiber angle on the epicardium
    beta_endo_lv=0,  # Sheet angle on the endocardium
    beta_epi_lv=0,  # Sheet angle on the epicardium
)

Calculating scalar fields
Compute scalar laplacian solutions with the markers: 
epi: 2
lv: 1
base: 7
  Num vertices: 149636
  Num cells: 734930
Solving linear variational problem.
  Apex coord: (45.82, -26.93, -38.51)
Solving linear variational problem.
Solving Laplace equation
 epi = 1, lv = 0
Solving linear variational problem.
 lv = 1, epi = 0
Solving linear variational problem.

Calculating gradients
Compute fiber-sheet system
Angles: 
alpha: 
 endo_lv: 60
 epi_lv: -60
 endo_septum: 60
 epi_septum: -60
 endo_rv: 60
 epi_rv: -60
beta: 
 endo_lv: 0
 epi_lv: 0
 endo_septum: 0
 epi_septum: 0
 endo_rv: 0
 epi_rv: 0


In [22]:
new_lv = LV.from_nodes_elements(mesh.coordinates(), mesh.cells())

In [56]:
fiber_pts_vec = fiber.compute_vertex_values().reshape((3,-1)).T
sheet_pts_vec = sheet.compute_vertex_values().reshape((3,-1)).T
sheet_normal_pts_vec = sheet_normal.compute_vertex_values().reshape((3,-1)).T

new_lv.mesh.point_data["fiber_pts_vec"] = fiber_pts_vec
new_lv.mesh.point_data["sheet_pts_vec"] = sheet_pts_vec
new_lv.mesh.point_data["sheet_normal_pts_vec"] = sheet_normal_pts_vec

In [24]:
new_lv.mesh.save("lvtetmesh_with_fibers.vtk")

In [102]:
import dolfin
import meshio

with dolfin.XDMFFile(mesh.mpi_comm(), "lvtetmesh_fiber.xdmf") as xdmf:
    xdmf.write(fiber)
meshio_mesh = meshio.read("lvtetmesh_fiber.xdmf")
meshio_mesh.write("lvtetmesh_fiber.vtk")

with dolfin.XDMFFile(mesh.mpi_comm(), "lvtetmesh_sheet.xdmf") as xdmf:
    xdmf.write(sheet)
meshio_mesh = meshio.read("lvtetmesh_sheet.xdmf")
meshio_mesh.write("lvtetmesh_sheet.vtk")

with dolfin.XDMFFile(mesh.mpi_comm(), "lvtetmesh_sheet_normal.xdmf") as xdmf:
    xdmf.write(sheet_normal)
meshio_mesh = meshio.read("lvtetmesh_sheet_normal.xdmf")
meshio_mesh.write("lvtetmesh_sheet_normal.vtk")

In [103]:
new_mesh = pv.read("lvtetmesh_fiber.vtk")
fiber_pts_vec = new_mesh.point_data["fiber"]
new_mesh = pv.read("lvtetmesh_sheet.vtk")
sheet_pts_vec = new_mesh.point_data["sheet"]
new_mesh = pv.read("lvtetmesh_sheet_normal.vtk")
sheet_normal_pts_vec = new_mesh.point_data["sheet_normal"]

In [20]:
fiber_pts_vec

NameError: name 'fiber_pts_vec' is not defined

In [52]:
fiber_pts_vec.take(map_from_mesh_to_pts[:,1], axis=0)

array([[-0.9005333 , -0.12249891,  0.41717359],
       [-0.7344604 , -0.00426298, -0.67863816],
       [-0.30471146,  0.95238225,  0.01090754],
       ...,
       [ 0.01098969,  0.21599804,  0.97633195],
       [ 0.08162461,  0.97255434,  0.21788869],
       [-0.82022178,  0.47488642, -0.31893434]])

In [ ]:
fiber_pts_vec_mapped = fiber_pts_vec

In [57]:
lv.mesh.point_data["fiber_pts_vec"] = fiber_pts_vec.take(map_from_mesh_to_pts[:,1], axis=0)
lv.mesh.point_data["sheet_pts_vec"] = sheet_pts_vec.take(map_from_mesh_to_pts[:,1], axis=0)
lv.mesh.point_data["sheet_normal_pts_vec"] = sheet_normal_pts_vec.take(map_from_mesh_to_pts[:,1], axis=0)

In [60]:
lv.mesh.save("lvtetmesh_with_fibers_2.vtk")

In [85]:
fiber(lv.mesh.points[0])

array([-0.67264064, -0.31638671,  0.66892004])

In [42]:
fiber.compute_vertex_values(mesh)

array([-0.80566354, -0.80468203, -0.78885649, ..., -0.52032557,
        0.85300445,  0.54124435])

In [25]:
fiber_pts_vec = fiber.compute_vertex_values().reshape((3,-1)).T
sheet_pts_vec = sheet.compute_vertex_values().reshape((3,-1)).T
sheet_normal_pts_vec = sheet_normal.compute_vertex_values().reshape((3,-1)).T

In [26]:
lv.mesh.point_data["fiber_pts_vec"] = fiber_pts_vec
lv.mesh.point_data["sheet_pts_vec"] = sheet_pts_vec
lv.mesh.point_data["sheet_normal_pts_vec"] = sheet_normal_pts_vec

In [27]:
lv.mesh.save("lvtetmesh_with_fibers.vtk")

In [38]:
np.linalg.norm(sheet_normal_pts_vec[1000])

1.0